In [2]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 4])

In [3]:
data.validate(raise_on_error=True)

True

In [4]:
print(data.keys())

['edge_index', 'x']


In [5]:
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [6]:
for key, item in data:
    print(f'{key} found in data')

x found in data
edge_index found in data


In [7]:
'edge_attr' in data

False

In [8]:
data.num_nodes

3

In [9]:
data.has_isolated_nodes()

False

In [10]:
data.has_self_loops()

False

In [13]:
device = torch.device('cuda')
data = data.to(device)
data

Data(x=[3, 1], edge_index=[2, 4])

In [15]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
len(dataset)

600

In [17]:
data = dataset[0]
data

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [19]:
train_dataset = dataset[:540]
train_dataset

ENZYMES(540)

In [20]:
dataset.num_classes

6

In [21]:
dataset.num_node_features

3

In [28]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
from torch_geometric.datasets import Planetoid
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [30]:
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.loader import NeighborLoader
from tqdm import tqdm

# 1. 数据（单图，节点分类）
dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# 2. 定义 GCN（节点分类，不需要池化）
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)

model = GCN(dataset.num_features, 64, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# 3. 建 NeighborLoader（train/val/test 三个 loader）
train_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10],          # 每一层采样的邻居数
    batch_size=1024,
    input_nodes=data.train_mask,     # 只从训练节点上采样 batch
    shuffle=True
)

# 验证/测试通常全图跑一次最简单；如果图太大，也可以再做推理采样
# 这里为了简单起见，直接全图测试
@torch.no_grad()
def full_batch_eval(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=-1)
    correct = int((pred[mask] == data.y[mask]).sum())
    total = int(mask.sum())
    return correct / total

# 4. 训练（mini-batch 采样）
def train():
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Train"):
        # 注意：NeighborLoader 返回的 batch 是一个子图，它有自己的 x / edge_index / n_id 等
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)

        # batch 中的目标节点（对应 input_nodes）位于 batch.n_id[:batch.batch_size]
        # 新版 PyG 会在 batch 中直接给出 "batch.batch_size"（>=2.3.0），
        # 如果没有，则可保存 input_id 索引手动截取。这里采用通用写法：
        target_idx = batch.input_id  if hasattr(batch, 'input_id')  else batch.n_id[:batch.batch_size]

        loss = F.nll_loss(out[target_idx], data.y[target_idx])
        loss.backward()
        optimizer.step()
        total_loss += float(loss)
    return total_loss / len(train_loader)

# 5. 训练 + 测试
for epoch in range(1, 201):
    loss = train()
    train_acc = full_batch_eval(data.train_mask)
    val_acc   = full_batch_eval(data.val_mask)
    test_acc  = full_batch_eval(data.test_mask)
    if epoch % 10 == 0 or epoch == 1:
        print(f'Epoch {epoch:03d} | Loss {loss:.4f} | '
              f'Train {train_acc:.4f} | Val {val_acc:.4f} | Test {test_acc:.4f}')

Processing...
Done!
/home/rtx4090/anaconda3/lib/python3.12/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
Train:   0%|          | 0/1 [00:00<?, ?it/s]


ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.loader import NeighborLoader
from tqdm import tqdm

# 1. 数据（单图，节点分类）
dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# 2. 定义 GCN（节点分类，不需要池化）
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)

model = GCN(dataset.num_features, 64, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# 3. 建 NeighborLoader（train/val/test 三个 loader）
train_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10],          # 每一层采样的邻居数
    batch_size=1024,
    input_nodes=data.train_mask,     # 只从训练节点上采样 batch
    shuffle=True
)

# 验证/测试通常全图跑一次最简单；如果图太大，也可以再做推理采样
# 这里为了简单起见，直接全图测试
@torch.no_grad()
def full_batch_eval(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=-1)
    correct = int((pred[mask] == data.y[mask]).sum())
    total = int(mask.sum())
    return correct / total

# 4. 训练（mini-batch 采样）
def train():
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Train"):
        # 注意：NeighborLoader 返回的 batch 是一个子图，它有自己的 x / edge_index / n_id 等
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)

        # batch 中的目标节点（对应 input_nodes）位于 batch.n_id[:batch.batch_size]
        # 新版 PyG 会在 batch 中直接给出 "batch.batch_size"（>=2.3.0），
        # 如果没有，则可保存 input_id 索引手动截取。这里采用通用写法：
        target_idx = batch.input_id  if hasattr(batch, 'input_id')  else batch.n_id[:batch.batch_size]

        loss = F.nll_loss(out[target_idx], data.y[target_idx])
        loss.backward()
        optimizer.step()
        total_loss += float(loss)
    return total_loss / len(train_loader)

# 5. 训练 + 测试
for epoch in range(1, 201):
    loss = train()
    train_acc = full_batch_eval(data.train_mask)
    val_acc   = full_batch_eval(data.val_mask)
    test_acc  = full_batch_eval(data.test_mask)
    if epoch % 10 == 0 or epoch == 1:
        print(f'Epoch {epoch:03d} | Loss {loss:.4f} | '
              f'Train {train_acc:.4f} | Val {val_acc:.4f} | Test {test_acc:.4f}')

Processing...
Done!
/home/rtx4090/anaconda3/lib/python3.12/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
Train:   0%|          | 0/1 [00:00<?, ?it/s]
